# Sistema de Recomendación de Videojuegos

## Importar Librerías

In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import os
import sys

# Ignorar advertencias
warnings.filterwarnings('ignore')

### importar funciones personalizadas de ../functions/EDA.py

In [15]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

In [16]:
# importar función personalizada de ./VideoGameRecommender/functions/EDA.py
# Esta función permite traer el archivo en el formato que se encuentre (CSV o Parquet)
from functions.EDA import get_file

### Cargar DataSets

In [28]:
# Se importa el dataset con la función get_file() del archivo ../functions/EDA.py
df = get_file('dummies')
games = get_file('games')
df.head(3)

,6dof,action,adventure,agriculture,animation,anime,arcade,arena,audio,board,...,thriller,top-down,tower defense,trains,turn-based,utilities,video,visual novel,war,wrestling
0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Escalar Datos

In [18]:
# Inicializar escalador
scaler = StandardScaler()

In [19]:
# se guardan los nombres de las columnas
cols = df.columns
# obtener los ids

In [20]:
# se escala la data en otro df
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data)

scaled_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,-0.030252,1.234151,-0.644272,-0.037673,-0.090065,-0.222865,-0.179669,-0.059437,-0.061343,-0.090323,...,-0.070114,-0.115103,10.181135,-0.085028,-0.212026,-0.137178,-0.064636,-0.13975,-0.199596,-0.019128
1,-0.030252,1.234151,1.552138,-0.037673,-0.090065,-0.222865,-0.179669,-0.059437,-0.061343,-0.090323,...,-0.070114,-0.115103,-0.098221,-0.085028,-0.212026,-0.137178,-0.064636,-0.13975,-0.199596,-0.019128
2,-0.030252,1.234151,-0.644272,-0.037673,-0.090065,-0.222865,-0.179669,-0.059437,-0.061343,-0.090323,...,-0.070114,-0.115103,-0.098221,-0.085028,-0.212026,-0.137178,-0.064636,-0.13975,5.010133,-0.019128


In [22]:
# calcular matriz de similitud
matrix = cosine_similarity(scaled_df)

In [73]:
def get_recommendations(item_id, similarity_matrix, items, top_n=5):
    ''' 
    Esta función recibe el id de un juego, la matriz de similitud de coseno, items y la cantidad de juegos que se deseen recomendar y retorna esa cantidad de elementos.
    '''
    # Obtiene el índice del ítem
    idx = items.index[items['id'] == item_id].tolist()[0]
    
    # Obtiene las similitudes para el ítem especificado
    sim_scores = list(enumerate(similarity_matrix[idx]))
    
    # Ordena los ítems basados en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Selecciona los top_n ítems más similares
    sim_scores = sim_scores[1:top_n + 1]  # Omite el primer elemento (el mismo ítem)
    
    # Obtiene los índices de los ítems
    item_indices = [i[0] for i in sim_scores]
    
    # Retorna los ítems recomendados
    return items.iloc[item_indices]['id']

In [78]:
def decode_game(id):
    # Filtrar el DataFrame por ID
    game = games[games['id'] == id]

    if game.empty:
        return f"No se encontró un juego con el ID {id}"

    # seleccionar solo una de las copias de id
    res = game.iloc[0, 1] + " (" + game.iloc[0, 2] + ")"

    return pd.DataFrame(res)

In [79]:
# se prueba la función con el id 230410 (Warframe)
id = 230410

recommendations = get_recommendations(id, matrix, df, top_n=5)
recommendations['id'].apply(decode_game)

ValueError: DataFrame constructor not properly called!